# Exercise 1 -- Earth Science dataset

### 1.	Create a regression model to predict global average temperature based on CO2, N2O, and CH4. Uss KNN algorithm. 

some steps are given to show how you can visulazie data to helo speed up the process

In [ ]:
import time
start = time.time()

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

# -- Extend cell width

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# -- load data

data = pd.read_csv('C:/Users/mxp180004/Downloads/World_Climate.csv')

N2O, Global atmospheric nitrous oxide concentration
Atmospheric nitrous oxide (N₂O) concentration is measured in parts per billion (ppb).

Monthly sea surface temperature anomaly
This is measured at a nominal depth of 20cm, and given relative to the average temperature from the period of 1961 - 1990.

Sea Surface anomaly
This is measured at a nominal depth of 20cm, and given relative to the average temperature from the period of 1961 - 1990.

Temperature anomaly
The monthly combined land-surface air and sea-surface water temperature anomaly is given as the deviation from the1951–1980 mean.


CH4, Global atmospheric methane concentration
Atmospheric methane (CH₄) concentration is measured in parts per billion (ppb).

CO2, Global atmospheric CO₂ concentration
Atmospheric carbon dioxide (CO₂) concentration is measured in parts per million (ppm).


Sea level rise
Global mean sea level rise is measured relative to the 1993 - 2008 average sea level. This is shown as three series: the widely-cited Church & White
dataset; the University of Hawaii Sea Level Center (UHLSC); and the average of the two.

Ocean acidification: mean seawater PH
Mean seawater pH is shown based on in-situ measurements of pH from the Aloha station in Hawaii.

In [ ]:
data.info()

In [ ]:
! pip install missingno

In [ ]:
# available data is shown in gray color and missing data points are in white for each column.
import missingno as msno
msno.matrix(data)

In [ ]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# create data
values=data['monthly_sea_surface_temperature_anomaly'].dropna()
values.plot(figsize=(20,10))

In [ ]:
# if you want plot 2 or more in one graph
value1 =data['CO2 Monthly averaged'].dropna()
value2 =data['CO2 Annual averaged'].dropna()
value12 = pd.concat([value1, value2], axis=1)
value12.plot(figsize=(20,10))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# -- heat map to see corrolation between features
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(30,30))

# -- plot heat map
sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn",) 

In [ ]:
df=data[['Date','Temperature anomaly','N2O Monthly averaged','CH4 Monthly averaged','CO2 Monthly averaged']].dropna()
X=df[['CO2 Monthly averaged','N2O Monthly averaged','CH4 Monthly averaged']]
y=df['Temperature anomaly']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# -- heat map to see corrolation between features
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(10,10))

# -- plot heat map
sns.heatmap(data[top_corr_features].corr(),annot=True,cmap="RdYlGn",) 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

cols_to_drop = []
num_scalers = {}
'''Scale only original columns'''
for col in X:
 if pd.api.types.is_numeric_dtype(X[col].dtype):
    print("MinMax scale of ", col)
    scaler = MinMaxScaler()
    scaler.fit(X[[col]])
    X[col+"_sc"] = scaler.transform(X[[col]])
    
    num_scalers[col] = [deepcopy(scaler),"MinMax"]
    cols_to_drop.append(col)
X=X.drop(columns=cols_to_drop)

In [ ]:
# -- split data 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline
train_score_array = []
test_score_array = []

for k in range(1,25):
    knn_reg = KNeighborsRegressor(k)
    knn_reg.fit(X_train, y_train)
    train_score_array.append(knn_reg.score(X_train, y_train))
    test_score_array.append(knn_reg.score(X_test, y_test))

x_axis = range(1,25)
plt.subplots(figsize = (20,5))
plt.plot(x_axis, train_score_array, c = 'g', label = 'Train Score')
plt.plot(x_axis, test_score_array, c = 'b', label = 'Test Score')
plt.legend()
plt.xlabel('n_neighbors')
plt.ylabel('MSE')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline
train_score_array = []
test_score_array = []

knn_reg = KNeighborsRegressor(6)
knn_reg.fit(X_train, y_train)
print('train score: ', knn_reg.score(X_train, y_train))
print('test  score: ',knn_reg.score(X_test, y_test))


### 2.	If the amount of CO2, N2O, and CH4 decrease gradually by 0.5%,0.1%, and 0.1% respectively every year for the next 15 years, what is the model prediction for reduction of the global average temperature in the next 15 years? Plot the results.

In [ ]:
# -- load data

data_future = pd.read_csv('C:/Users/mxp180004/Downloads/World_Future_Climate.csv')

In [ ]:
data_future

In [ ]:
future=pd.DataFrame()
future['N2O Annual averaged']=data_future['N2O']
future['CH4 Annual averaged']=data_future['CH4']
future['CO2 Annual averaged']=data_future['CO2']
future

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
cols_to_drop = []
num_scalers = {}
'''Scale only original columns'''
for col in future:
  if pd.api.types.is_numeric_dtype(future[col].dtype):
    print("MinMax scale of ", col)
    scaler = MinMaxScaler()
    scaler.fit(future[[col]])
    future[col+"_sc"] = scaler.transform(future[[col]])
    
    num_scalers[col] = [deepcopy(scaler),"MinMax"]
    cols_to_drop.append(col)
future=future.drop(columns=cols_to_drop)
future

In [ ]:
predict = knn_reg.predict(future)
predict

In [ ]:
data_future['Tempreture anomaly predict']=predict
data_future

In [ ]:
# fix date columns

data['Date'] = pd.to_datetime(data['Date'])
data_future['date'] = pd.to_datetime(data_future['date'], format='%Y')

# plot
data_plot=data[['Date','Temperature anomaly']].dropna()
plt.rcParams.update({'font.size': 22})




plt.figure(figsize=(30,10)) 
plt.plot('Date', 'Temperature anomaly', data=data_plot, linestyle='-', marker='o')
plt.plot('date', 'Tempreture anomaly predict', data=data_future, linestyle='-', marker='o',color='mediumvioletred')
plt.show()